In [1]:
!pip install xlwt
!pip install openpyxl
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Requirement already up-to-date: google-api-python-client in c:\users\masaki\anaconda3\lib\site-packages (1.9.3)
Requirement already up-to-date: google-auth-httplib2 in c:\users\masaki\anaconda3\lib\site-packages (0.0.3)
Requirement already up-to-date: google-auth-oauthlib in c:\users\masaki\anaconda3\lib\site-packages (0.4.1)


In [2]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import pandas as pd

In [3]:
# スクレイピング対象の URL にリクエストを送り HTML を取得する
r = requests.get('https://www.amazon.co.jp/gp/offer-listing/B07XV8VSZT/ref=dp_olp_new?ie=UTF8&condition=new') 
#r.raise_for_status()#文字化け対策
#r.encoding = 'shift-jis'  # 文字コード
# レスポンスの HTML から BeautifulSoup オブジェクトを作る
soup = BeautifulSoup(r.text, 'html.parser')
print(soup.prettify())#HTMLの整列

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   ²Àfð¨©¯µÄ¢Ü·I
  </title>
  <style type="text/css">
   <!--
.serif { font-family: times,serif; font-size: medium; }
.sans { font-family: osaka,MS PSVbN,arial,helvetica,sans-serif; font-size: medium; }
.small { font-family: osaka,MS PSVbN,arial,helvetica,sans-serif; font-size: small; }
.h1 { font-family: osaka,MS PSVbN,arial,helvetica,sans-serif; color: #CC6600; font-size: medium; }
.h3color { font-family: osaka,MS PSVbN,arial,helvetica,sans-serif; color: #CC6600; font-size: small; }
.tiny { font-family: osaka,MS PSVbN,arial,helvetica,sans-serif; font-size: x-small; }
-->
  </style>
 </head>
 <body alink="#FF9933" bgcolor="#FFFFFF" link="#003399" text="#000000" vlink="#996633">
  <!--
        To discuss automated access to Amazon data please contact api-services-support@amazon.com.
        For information about migrating to our APIs refer to our Marketplace 

In [4]:
#データフレームの用意
columns=["price","delivery fee","condition","company"]
df=pd.DataFrame(columns=columns)

In [5]:
div = soup.find_all("div", class_="a-row a-spacing-mini olpOffer")#各会社ごとの情報抜き出し
#print(div)

In [6]:
company=[]
price=[]
delivery_fee=[]
condition=[]
for i in range(len(div)):
    info = div[i].find_all("span", class_="a-offscreen")
    info = info[0].string.rsplit('、',1)
    com=info[0].split('：')
    pri_string=info[1].split('￥ ')
    pri=int(pri_string[1].replace(',', ''))
    company.append(com[1])
    price.append(pri)
    delivery_fee_info = div[i].find_all("span", class_="olpShippingPrice")
    condition_info = div[i].find_all("span", class_="a-size-medium olpCondition a-text-bold")
    #中古品か新品か判別
    if '中古品' in condition_info[0].string.strip():
        condition.append("中古品")
    else:
        condition.append(condition_info[0].string.strip())
    #無料か配送料が必要か判別
    if len(delivery_fee_info)==0:
        delivery_fee.append("無料")
    else:
        delivery_fee.append(delivery_fee_info[0].string)

In [7]:
print(price)

[]


In [8]:
df["price"]=price
df["delivery fee"]=delivery_fee
df["condition"]=condition
df["company"]=company
print(df)

Empty DataFrame
Columns: [price, delivery fee, condition, company]
Index: []


In [ ]:
from email.mime.text import MIMEText
from email.utils import formatdate
from email.header import Header
import smtplib
import datetime

In [9]:
from_addr = "fpspro3758juzu@gmail.com"
to_addr = "fpspro3758juzu@gmail.com"
subject = u"リングフィットの値段"
dt_now = datetime.datetime.now()
head = "この価格情報はAmazonから収集しています。\n収集時間は"+str(dt_now)+"です。\n\n"
body1 ="最安値は ￥"+ str(df["price"][0])+"\n状態:"+str(df["condition"][0])+"\n送料:"+str(df["delivery fee"][0])+" \n"+str(df["company"][0])+"\n\n"
body2 ="2番目は ￥"+ str(df["price"][1])+"\n状態:"+str(df["condition"][1])+"\n送料:"+str(df["delivery fee"][1])+" \n"+str(df["company"][1])+"\n\n"
body3 ="3番目は ￥"+ str(df["price"][2])+"\n状態:"+str(df["condition"][2])+"\n送料:"+str(df["delivery fee"][2])+" \n"+str(df["company"][2])+"\n\n"
body4 ="4番目は ￥"+ str(df["price"][3])+"\n状態:"+str(df["condition"][3])+"\n送料:"+str(df["delivery fee"][3])+" \n"+str(df["company"][3])+"\n\n"
body5 ="5番目は ￥"+ str(df["price"][4])+"\n状態:"+str(df["condition"][4])+"\n送料:"+str(df["delivery fee"][4])+" \n"+str(df["company"][4])+"\n\n"
body6 ="6番目は ￥"+ str(df["price"][5])+"\n状態:"+str(df["condition"][5])+"\n送料:"+str(df["delivery fee"][5])+" \n"+str(df["company"][5])+"\n\n"

body = head + body1 + body2 + body3+ body4+ body5+ body6

IndexError: index out of bounds

In [ ]:
def create_message(from_addr, to_addr, subject, body):
    msg = MIMEText(body,'plain','utf-8')
    msg['Subject'] = Header(subject, 'utf-8')
    msg['From'] = from_addr
    msg['To'] = to_addr
    msg['Date'] = formatdate()
    return msg

In [ ]:
def send_mail(from_addr, to_addr, body):
    smtpobj = smtplib.SMTP('smtp.gmail.com', 587)
    smtpobj.ehlo()
    smtpobj.starttls()
    smtpobj.ehlo()
    smtpobj.login(from_addr, "fpqqiajfrnyzoger")
    smtpobj.sendmail(from_addr, to_addr, body.as_string())
    smtpobj.close()

In [ ]:
if df["price"][0]<8000:
    print("今が買い時")
else:
    print("まだ高い")

In [ ]:
#gmailで送信
msg = create_message(from_addr, to_addr, subject, body)
send_mail(from_addr, to_addr, msg)